In [ ]:
!pip install tensorflow numpy pandas tqdm scikit-learn segmentation-models tensorflow-addons flammkuchen opencv-python-headless imageio

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import tensorflow as tf
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score

from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.data import load_data, metric_mape, mape_ap, mape_pp
from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.Callbacks import get_callbacks
from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Models.GlottisNetV2_e import glottisnetV2_e


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Segmentation Models: using `tf.keras` framework.


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [7]:
import cv2
import imageio as io
from tensorflow.keras.models import load_model
import flammkuchen as fl
from google.colab.patches import cv2_imshow
import tensorflow_addons as tfa

def extract_points_from_heatmap(heatmap, original_size):
    points = []
    resized_size = (512,256) # Dimensiones que recibe el modelo
    for i in range(heatmap.shape[-1]):  # Asumimos dos puntos: anterior y posterior
        # Extrae las coordenadas de los puntos de máxima activación
        y, x = np.unravel_index(np.argmax(heatmap[..., i]), heatmap[..., i].shape)

        # Escala las coordenadas de vuelta al tamaño original de la imagen
        scale_x = original_size[1] / resized_size[1]
        scale_y = original_size[0] / resized_size[0]
        x = int(x * scale_x)
        y = int(y * scale_y)

        points.append((x, y))
    return points[0], points[1]

def draw_points_and_line(img, point1, point2, color_points=(0, 255, 255), color_line=(0, 255, 255), thickness=2):
    cv2.circle(img, point1, radius=5, color=color_points, thickness=-1)
    cv2.circle(img, point2, radius=5, color=color_points, thickness=-1)
    cv2.line(img, point1, point2, color=color_line, thickness=thickness)
    return img

def draw_contours_on_image(mask, img, contour_color=(255, 0, 0)):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, contour_color, 2)
    return img

In [8]:
def process_and_recreate_video(video_path, model_path, output_video_path):
    model = load_model(model_path, compile=False)

    ims = io.mimread(video_path, memtest=False)
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    original_size = (height, width)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (ims[0].shape[1], ims[0].shape[0]))

    for i in range(len(ims)):
        img_original = ims[i].astype(np.float32)
        img_gray = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_gray, (256, 512))

        normalizedImg = np.zeros(img_resized.shape)
        img_normalized = cv2.normalize(img_resized, normalizedImg, -1, 1, cv2.NORM_MINMAX)
        img_model_input = img_normalized[None, ..., None]

        points_pred, seg_pred = model.predict(img_model_input)
        mask = np.asarray(np.squeeze(seg_pred))
        points_pred = np.asarray(np.squeeze(points_pred))


        point1, point2 = extract_points_from_heatmap(points_pred, original_size)

        mask_resized = cv2.resize(np.round(mask), (img_original.shape[1], img_original.shape[0])).astype(bool)

        if img_original.ndim == 2 or (img_original.ndim == 3 and img_original.shape[2] == 1):
            img_original = cv2.cvtColor(img_original, cv2.COLOR_GRAY2BGR)

        img_with_points_and_line = draw_points_and_line(img_original, point1, point2)
        img_with_contours = draw_contours_on_image(mask_resized.astype(np.uint8), img_with_points_and_line)


        video_writer.write(img_with_contours.astype(np.uint8))
    video_writer.release()
    cap.release()


In [ ]:
model_path = 'drive/MyDrive/Glottis/GlottisNetV2e/models/steps/epoch025.h5'
video_path = '/content/drive/MyDrive/Glottis/videos_VPLAB/FP007.avi'
output_video_path = '/content/drive/MyDrive/Glottis/videos_segmentados/FP007_seg.mp4'
process_and_recreate_video(video_path, model_path, output_video_path)

1/1 [==============================] - 1s 764ms/step


In [9]:
model_path = 'drive/MyDrive/Glottis/GlottisNetV2e/models/steps/epoch025.h5'
video_path = 'drive/MyDrive/Glottis/videos_BAGLS/40.mp4'
output_video_path = '/content/drive/MyDrive/Glottis/videos_segmentados/40_seg.mp4'
process_and_recreate_video(video_path, model_path, output_video_path)

1/1 [==============================] - 0s 481ms/step
